In [76]:
import pandas as pd

## Get the Data Generated from the Previous Module

This dataset, derived from the gpt4o mini, includes fields such as 'answer_llm', 'answer_orig', 'document', 'question', and 'course'.

- 'answer_llm': The response provided by the language model to the generated question.
- 'answer_orig': The original answer from the ground truth dataset, which was used to generate the corresponding question.
- 'document': The identifier for the document.
- 'question': The question generated from the original answer.
- 'course': The related course.

The dataset was compiled by processing a ground truth dataset that contains questions, associated courses, and document IDs. This ground truth dataset was produced from the original set of FAQ documents. For each answer in these documents, an LLM generated five related questions, recording the document ID linked to each question.

Using the fields 'answer_llm' and 'answer_orig', we can compute the cosine similarity to assess the accuracy of the LLM's responses to the generated questions.

In [77]:
github_url = 'https://github.com/DataTalksClub/llm-zoomcamp/blob/main/04-monitoring/data/results-gpt4o-mini.csv'
url = f'{github_url}?raw=1'
df = pd.read_csv(url)

In [78]:
df = df.iloc[:300] #only use the first 300 documents

## Get the embeddings model

multi-qa-mpnet-base-dot-v1 from the Sentence Transformer library

In [79]:
from sentence_transformers import SentenceTransformer
model_name = 'multi-qa-mpnet-base-dot-v1'
embedding_model = SentenceTransformer(model_name)

/home/ubuntu/.local/share/virtualenvs/llm-homework-04-4mt2ev0V/lib/python3.9/site-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


## Create the embeddings for the first LLM answer

In [80]:
answer_llm = df.iloc[0].answer_llm
print(answer_llm)

You can sign up for the course by visiting the course page at [http://mlzoomcamp.com/](http://mlzoomcamp.com/).


In [81]:
first_answer_embedding = embedding_model.encode(answer_llm)
print(first_answer_embedding[0])

-0.42244658


## Compute Dot Product

Now for each answer pair, let's create embeddings and compute dot product between them

We will put the results (scores) into the `evaluations` list

What's the 75% percentile of the score?

In [82]:
from tqdm.auto import tqdm

In [83]:
import numpy as np

In [84]:
evaluations = []

for index, row in tqdm(df.iterrows(), total=df.shape[0]):
    v_answer_llm = embedding_model.encode(row['answer_llm']) 
    v_answer_orig = embedding_model.encode(row['answer_orig'])
    score = v_answer_llm.dot(v_answer_orig)
    evaluations.append(score)


100%|██████████| 300/300 [01:04<00:00,  4.64it/s]


In [85]:
percentile_75 = np.percentile(evaluations, 75)

print("The 75th percentile is:", percentile_75)

The 75th percentile is: 31.674313


## The results are not within the range [0,1]. We need to normalize the vectors

In [86]:
def compute_norm(v):
    norm = np.sqrt((v * v).sum())
    v_norm = v / norm
    return v_norm

In [87]:
def evaluate(df):
    evaluations = []
    for index, row in tqdm(df.iterrows(), total=df.shape[0]):
        v_answer_llm = compute_norm(embedding_model.encode(row['answer_llm'])) 
        v_answer_orig = compute_norm(embedding_model.encode(row['answer_orig']))
        score = v_answer_llm.dot(v_answer_orig)
        evaluations.append(score)
    return evaluations

In [88]:
evaluations = evaluate(df)

100%|██████████| 300/300 [01:04<00:00,  4.64it/s]


In [89]:
percentile_75 = np.percentile(evaluations, 75)

print("The 75th percentile is:", percentile_75)

The 75th percentile is: 0.8362348


## Now we will use an alternative metric - the ROUGE score

This is a set of metrics that compares two answers based on the overlap of n-grams, word sequences, and word pairs.

It can give a more nuanced view of text similarity than just cosine similarity alone.

Let's compute the ROUGE score between the answers at the index 10 of our dataframe (doc_id=5170565b)

n-grams are contiguous sequence of n items from a given sample of text or speech. These items can be phonemes, syllables, letters, words, or base pairs according to the level of textual analysis being conducted.

In [90]:
from rouge import Rouge
rouge_scorer = Rouge()

In [91]:
r = df.iloc[10]

In [92]:
scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]

There are three scores: rouge-1, rouge-2 and rouge-l, and precision, recall and F1 score for each.

- rouge-1 - the overlap of unigrams,
- rouge-2 - bigrams,
- rouge-l - the longest common subsequence

In [93]:
scores

{'rouge-1': {'r': 0.45454545454545453,
  'p': 0.45454545454545453,
  'f': 0.45454544954545456},
 'rouge-2': {'r': 0.21621621621621623,
  'p': 0.21621621621621623,
  'f': 0.21621621121621637},
 'rouge-l': {'r': 0.3939393939393939,
  'p': 0.3939393939393939,
  'f': 0.393939388939394}}

## Average rouge score

Let's compute the average F-score between rouge-1, rouge-2 and rouge-l for the same record

In [94]:
# Extract F-scores using list comprehension
f_scores = [scores[key]['f'] for key in scores]

# Calculate the average F-score using NumPy
average_f_score = np.mean(f_scores)

print("The average F-score is:", average_f_score)

The average F-score is: 0.35490034990035496


## Average rouge score for all the data points

Now let's compute the F-score for all the records and create a dataframe from them.

What's the average F-score in rouge_2 across all the records?

In [95]:
# Create an empty DataFrame with predefined columns
f_scores_df = pd.DataFrame(columns=['rouge-1_f', 'rouge-2_f', 'rouge-l_f'])

In [96]:
rows_list = []  # List to collect each row dictionary

for index, r in tqdm(df.iterrows(), total=df.shape[0]):
    # Compute scores
    scores = rouge_scorer.get_scores(r['answer_llm'], r['answer_orig'])[0]
    
    # Extract and store F-scores
    f_score_record = {
        'rouge-1_f': scores['rouge-1']['f'],
        'rouge-2_f': scores['rouge-2']['f'],
        'rouge-l_f': scores['rouge-l']['f']
    }
    
    # Append the record to the list
    rows_list.append(f_score_record)

# Create DataFrame from the list of dictionaries
f_scores_df = pd.DataFrame(rows_list)

100%|██████████| 300/300 [00:01<00:00, 262.99it/s]


In [97]:
# Now, calculate the average F-score for 'rouge-2'
average_rouge_2_f = f_scores_df['rouge-2_f'].mean()
print("The average F-score for ROUGE-2 across all records is:", average_rouge_2_f)

The average F-score for ROUGE-2 across all records is: 0.20696501983423318
